In [1]:
from PIL import Image

# 打开图片
image = Image.open("logo.png")  # 替换为你的图片路径

# 将图片转换为RGBA模式，以确保我们能处理透明度
image = image.convert("RGBA")

# 加载图片数据
data = image.getdata()

# 创建一个新的图像数据列表
new_data = []
for item in data:
    # 如果像素是透明的，则保留原样
    if item[3] == 0:
        new_data.append(item)
    # 如果像素不透明，则将其变为白色
    else:
        new_data.append((255, 255, 255, 255))

# 将新的图片数据写入图片
image.putdata(new_data)

# 保存或显示图片
image.save("white_logo.png")  # 替换为你想要保存的路径
image.show()